# Single Cell Quantification

-Single cell quantification with skimage

In [1]:
# HDI quantification
# import custom modules
from miaaim.quant.quantification import HDIquantification

# import external modules
from pathlib import Path
import pandas as pd
import os
import numpy as np
import skimage.io
import nibabel as nib
from tqdm import tqdm

import skimage.filters.edges as ed
import scipy.ndimage
from miaaim.io.imread import _import

# check versions and environment executable
import sys
sys.version
sys.executable

'/opt/conda/envs/miaaim-dev/bin/python'

In [2]:
# n-dimensional filter weights
SOBEL_EDGE = np.array([1, 0, -1])
SOBEL_SMOOTH = np.array([1, 2, 1]) / 4

def sobel(image, mask=None, *, axis=None, mode='reflect', cval=0.0):
    """Find edges in an image using the Sobel filter.
    Parameters
    ----------
    image : array
        The input image.
    mask : array of bool, optional
        Clip the output image to this mask. (Values where mask=0 will be set
        to 0.)
    axis : int or sequence of int, optional
        Compute the edge filter along this axis. If not provided, the edge
        magnitude is computed. This is defined as::
            sobel_mag = np.sqrt(sum([sobel(image, axis=i)**2
                                     for i in range(image.ndim)]) / image.ndim)
        The magnitude is also computed if axis is a sequence.
    mode : str or sequence of str, optional
        The boundary mode for the convolution. See `scipy.ndimage.convolve`
        for a description of the modes. This can be either a single boundary
        mode or one boundary mode per axis.
    cval : float, optional
        When `mode` is ``'constant'``, this is the constant used in values
        outside the boundary of the image data.
    Returns
    -------
    output : array of float
        The Sobel edge map.
    See also
    --------
    sobel_h, sobel_v : horizontal and vertical edge detection.
    scharr, prewitt, farid, skimage.feature.canny
    References
    ----------
    .. [1] D. Kroon, 2009, Short Paper University Twente, Numerical
           Optimization of Kernel Based Image Derivatives.
    .. [2] https://en.wikipedia.org/wiki/Sobel_operator
    Examples
    --------
    >>> from skimage import data
    >>> from skimage import filters
    >>> camera = data.camera()
    >>> edges = filters.sobel(camera)
    """
    output = ed._generic_edge_filter(image, smooth_weights=SOBEL_SMOOTH,
                                  axis=axis, mode=mode, cval=cval)
    output = ed._mask_filter_result(output, mask)
    return output


def sobel_h(image, mask=None):
    """Find the horizontal edges of an image using the Sobel transform.
    Parameters
    ----------
    image : 2-D array
        Image to process.
    mask : 2-D array, optional
        An optional mask to limit the application to a certain area.
        Note that pixels surrounding masked regions are also masked to
        prevent masked regions from affecting the result.
    Returns
    -------
    output : 2-D array
        The Sobel edge map.
    Notes
    -----
    We use the following kernel::
      1   2   1
      0   0   0
     -1  -2  -1
    """
    return sobel(image, mask=mask, axis=0)


def sobel_v(image, mask=None):
    """Find the vertical edges of an image using the Sobel transform.
    Parameters
    ----------
    image : 2-D array
        Image to process.
    mask : 2-D array, optional
        An optional mask to limit the application to a certain area.
        Note that pixels surrounding masked regions are also masked to
        prevent masked regions from affecting the result.
    Returns
    -------
    output : 2-D array
        The Sobel edge map.
    Notes
    -----
    We use the following kernel::
      1   0  -1
      2   0  -2
      1   0  -1
    """
    return sobel(image, mask=mask, axis=1)


def per_channel_sobel(arr, nms, sigma=2):
    """Sobel filter over each channel of 2D imaging data.

    Parameters
    ----------
    arr: numpy.ndarray with 3 axes.
        Input image consisting of c,y,x axes.

    nms: list
        Indicates names of markers in the array indexed in position 0.

    out_dir: os path
        Output directory to store resulting image.
        
    sigma: int
        Gaussian smoothing parameter.
    """
    # get sizes of image
    x,y,c = arr.shape
    # create array to store results
    grad_array = np.zeros((x,y,c,2))
    grad_mag_array = np.zeros((x,y,c))
    # create indexer
    i = 0
    # iterate through each gene
    for g in tqdm(nms, desc='Computing image gradients'):
        # get gene
        C_ = arr[:,:,i]
        # reshape
        C = C_.reshape(C_.shape[0],C_.shape[1])
        # apply gaussian smoothing
        if sigma is not None:
            # smooth
            C = scipy.ndimage.gaussian_filter(C,sigma=sigma)

        # compute gradient approximation with sobel filter
        grad_h = sobel_h(C)
        grad_v = sobel_v(C)
        # compute magnitude
        sobel_mag = np.sqrt(sum([i**2 for i in [grad_h,grad_v]])/2)

        # update output array with gradients
        grad_array[:,:,i,0] = grad_h
        grad_array[:,:,i,1] = grad_v
        # update output array of gradient magnitude
        grad_mag_array[:,:,i] = sobel_mag

        # update indexer
        i = i + 1     
        
    # return
    return grad_array, grad_mag_array, list(nms)

In [3]:
def TMAquantification(core_name, data_folder, gradients=True, grad_sigmas=[25]):
    # set path to core folder (root_folder below)
    core_folder = data_folder.joinpath(f"{core_name}")
    
    # check if segmentation directory exists
    _segdir = core_folder.joinpath("segmentation/imc/hdiseg-ilastik")
    # check if the registration transformix directory exists
    _transdir = core_folder.joinpath("registration/msi-imc")
    
    # check if segmentation exists
    if _segdir.exists():
        # set root folder as the core folder
        root_folder = core_folder
        # set path to segmentation mask
        masks = [ core_folder.joinpath(f"segmentation/imc/hdiseg-ilastik/{core_name}_core_probabilities_mask.tiff") ]
        # set path to imc preprocessed image to quantify
        imcIm = core_folder.joinpath(f"preprocessing/imc/{core_name}_core.ome.tiff")
        
        # add to list of images
        imagesList = [ imcIm ]
        # create markers list
        channelsList = [ data_folder.joinpath("imc-markers.csv") ]
        # create names list
        namesList = [ "imc" ]

        # create list to store gradient magnitude images in
        grad_mags_list = []
        # check if multimodal registered data exists
        if _transdir.exists():
            # set path to imc preprocessed image to quantify
            msiIm = root_folder.joinpath(f"registration/msi-imc/transformix/{core_name}_result.nii")        
            
            # update list with the msi data
            imagesList.append(msiIm)
            # update markers list
            channelsList.append(data_folder.joinpath("msi-markers.csv"))
            # update names list
            namesList.append("msi")

            # check if computing gradients
            if gradients:
                # read the registered msi data
                msiIm = _import.HDIreader(
                    path_to_data=msiIm,
                    path_to_markers=None,
                    flatten=False,
                    subsample=False,
                    method=None,
                    mask=None,
                    save_mem=False,
                    data=None,
                    image=None,
                    channels=None,
                    filename=None)
                # iterate over sigmas
                for s in grad_sigmas:
                    # compute gradients
                    grads, grad_magnitudes, _ = per_channel_sobel(
                        msiIm.hdi.data.image, 
                        nms=range(msiIm.hdi.data.image.shape[2]), 
                        sigma=s)
                    # create name for exporting gradint magnitudes
                    msiImGrad = root_folder.joinpath(f"registration/msi-imc/transformix/{core_name}_result_gradient_magnitude_{str(s)}.nii") 
                    # create nifti image
                    arr = nib.Nifti1Image(grad_magnitudes.transpose(1, 0, 2), affine=np.eye(4))
                    # export gradients
                    nib.save(arr,str(msiImGrad))
                    # update list with the msi data
                    imagesList.append(msiImGrad)
                    # update markers list
                    channelsList.append(data_folder.joinpath("msi-markers.csv"))
                    # update names list
                    namesList.append(f"msi-gradient-magnitude-sigma-{str(s)}")
                    # update list of gradient magnitude images
                    grad_mags_list.append(msiImGrad)

        # create quantification instance
        quantifier = HDIquantification(root_folder=root_folder,
                    masks=masks,
                    images=imagesList,       
                    channel_names=channelsList,
                    names=namesList,
                    probabilities_method="ilastik",
                    segmentation_method="hdiseg",
                    mask_props=None,
                    intensity_props=["intensity_mean"])
        # quantify
        quantifier.Quantify()
        # export documentation
        quantifier.QC()
        
        # remove the gradient images for storage space
        if len(grad_mags_list)>0:
            for z in grad_mags_list:
                z.unlink(missing_ok=False)

In [4]:
# set path to data folder
data_folder = Path("/opt/miaaim-20220912-TMA4/data")
# get list of all core names in the data folder
# check to make sure we have the core folders for only prostate samples which have
# multimodal data (here, indicated by /registration folder)
core_folders = [data_folder.joinpath(c) for c in os.listdir(data_folder) if data_folder.joinpath(c).is_dir()]
core_folders = [c for c in core_folders if Path(f"{c}/probabilities/imc/ilastik").is_dir()]

# print the size of the list of folders
print(f"NUMBER OF MASS CYTOMETRY CORES: {str(len(core_folders))}")
# iterate through data set and register
for c in core_folders:
    # register
    TMAquantification(core_name=c.name, data_folder=data_folder)

NUMBER OF MASS CYTOMETRY CORES: 61


Computing image gradients: 100%|██████████████████████████████████████████████████████| 702/702 [23:33<00:00,  2.01s/it]


MIAAIM QUANTIFICATION
MIAAIM VERSION 0.0.2
METHOD: HDIquantification
ROOT FOLDER: /opt/miaaim-20220912-TMA4/data/ROI010_PROSTATE_TMA001
PROVENANCE FOLDER: /opt/miaaim-20220912-TMA4/data/ROI010_PROSTATE_TMA001/docs/provenance
QC FOLDER: /opt/miaaim-20220912-TMA4/data/ROI010_PROSTATE_TMA001/docs/qc/quantification/hdiseg-ilastik 



PROCESSING DATA
Quantify: extracting single-cell measurements
Extracting single-cell data for /opt/miaaim-20220912-TMA4/data/ROI010_PROSTATE_TMA001/preprocessing/imc/ROI010_PROSTATE_TMA001_core.ome.tiff...
Finished 0
Finished 1
Finished 2
Finished 3
Finished 4
Finished 5
Finished 6
Finished 7
Finished 8
Finished 9
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Finished 28
Finished 29
Finished 30
Finished 31
Finished 32
Finished 33
Finished 34
Finished 35
Finished 36
Finished 37
Finished 38
Fini

INFO:root:Valid path...
INFO:root:Valid file extension...
INFO:root:file name: /opt/miaaim-20220912-TMA4/data/ROI011_PROSTATE_TMA002/registration/msi-imc/transformix/ROI011_PROSTATE_TMA002_result.nii
INFO:root:Finished parsing nifti
INFO:root:Finished
Computing image gradients: 100%|██████████████████████████████████████████████████████| 702/702 [19:23<00:00,  1.66s/it]


MIAAIM QUANTIFICATION
MIAAIM VERSION 0.0.2
METHOD: HDIquantification
ROOT FOLDER: /opt/miaaim-20220912-TMA4/data/ROI011_PROSTATE_TMA002
PROVENANCE FOLDER: /opt/miaaim-20220912-TMA4/data/ROI011_PROSTATE_TMA002/docs/provenance
QC FOLDER: /opt/miaaim-20220912-TMA4/data/ROI011_PROSTATE_TMA002/docs/qc/quantification/hdiseg-ilastik 



PROCESSING DATA
Quantify: extracting single-cell measurements
Extracting single-cell data for /opt/miaaim-20220912-TMA4/data/ROI011_PROSTATE_TMA002/preprocessing/imc/ROI011_PROSTATE_TMA002_core.ome.tiff...
Finished 0
Finished 1
Finished 2
Finished 3
Finished 4
Finished 5
Finished 6
Finished 7
Finished 8
Finished 9
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Finished 28
Finished 29
Finished 30
Finished 31
Finished 32
Finished 33
Finished 34
Finished 35
Finished 36
Finished 37
Finished 38
Fini

...

MIAAIM QUANTIFICATION
MIAAIM VERSION 0.0.2
METHOD: HDIquantification
ROOT FOLDER: /opt/miaaim-20220912-TMA4/data/ROI062_PROSTATE_TMA061
PROVENANCE FOLDER: /opt/miaaim-20220912-TMA4/data/ROI062_PROSTATE_TMA061/docs/provenance
QC FOLDER: /opt/miaaim-20220912-TMA4/data/ROI062_PROSTATE_TMA061/docs/qc/quantification/hdiseg-ilastik 



PROCESSING DATA
Quantify: extracting single-cell measurements
Extracting single-cell data for /opt/miaaim-20220912-TMA4/data/ROI062_PROSTATE_TMA061/preprocessing/imc/ROI062_PROSTATE_TMA061_core.ome.tiff...
Finished 0
Finished 1
Finished 2
Finished 3
Finished 4
Finished 5
Finished 6
Finished 7
Finished 8
Finished 9
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Finished 28
Finished 29
Finished 30
Finished 31
Finished 32
Finished 33
Finished 34
Finished 35
Finished 36
Finished 37
Finished 38
Fini

INFO:root:Valid path...
INFO:root:Valid file extension...
INFO:root:file name: /opt/miaaim-20220912-TMA4/data/ROI064_PROSTATE_Benign_TMA063N/registration/msi-imc/transformix/ROI064_PROSTATE_Benign_TMA063N_result.nii
INFO:root:Finished parsing nifti
INFO:root:Finished
Computing image gradients: 100%|██████████████████████████████████████████████████████| 702/702 [18:16<00:00,  1.56s/it]


MIAAIM QUANTIFICATION
MIAAIM VERSION 0.0.2
METHOD: HDIquantification
ROOT FOLDER: /opt/miaaim-20220912-TMA4/data/ROI064_PROSTATE_Benign_TMA063N
PROVENANCE FOLDER: /opt/miaaim-20220912-TMA4/data/ROI064_PROSTATE_Benign_TMA063N/docs/provenance
QC FOLDER: /opt/miaaim-20220912-TMA4/data/ROI064_PROSTATE_Benign_TMA063N/docs/qc/quantification/hdiseg-ilastik 



PROCESSING DATA
Quantify: extracting single-cell measurements
Creating directories for quantification export
Creating directories for quantification export
Extracting single-cell data for /opt/miaaim-20220912-TMA4/data/ROI064_PROSTATE_Benign_TMA063N/preprocessing/imc/ROI064_PROSTATE_Benign_TMA063N_core.ome.tiff...
Finished 0
Finished 1
Finished 2
Finished 3
Finished 4
Finished 5
Finished 6
Finished 7
Finished 8
Finished 9
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Fi

INFO:root:Valid path...
INFO:root:Valid file extension...
INFO:root:file name: /opt/miaaim-20220912-TMA4/data/ROI068_PROSTATE_TMA066/registration/msi-imc/transformix/ROI068_PROSTATE_TMA066_result.nii
INFO:root:Finished parsing nifti
INFO:root:Finished
Computing image gradients: 100%|██████████████████████████████████████████████████████| 702/702 [19:27<00:00,  1.66s/it]


MIAAIM QUANTIFICATION
MIAAIM VERSION 0.0.2
METHOD: HDIquantification
ROOT FOLDER: /opt/miaaim-20220912-TMA4/data/ROI068_PROSTATE_TMA066
PROVENANCE FOLDER: /opt/miaaim-20220912-TMA4/data/ROI068_PROSTATE_TMA066/docs/provenance
QC FOLDER: /opt/miaaim-20220912-TMA4/data/ROI068_PROSTATE_TMA066/docs/qc/quantification/hdiseg-ilastik 



PROCESSING DATA
Quantify: extracting single-cell measurements
Extracting single-cell data for /opt/miaaim-20220912-TMA4/data/ROI068_PROSTATE_TMA066/preprocessing/imc/ROI068_PROSTATE_TMA066_core.ome.tiff...
Finished 0
Finished 1
Finished 2
Finished 3
Finished 4
Finished 5
Finished 6
Finished 7
Finished 8
Finished 9
Finished 10
Finished 11
Finished 12
Finished 13
Finished 14
Finished 15
Finished 16
Finished 17
Finished 18
Finished 19
Finished 20
Finished 21
Finished 22
Finished 23
Finished 24
Finished 25
Finished 26
Finished 27
Finished 28
Finished 29
Finished 30
Finished 31
Finished 32
Finished 33
Finished 34
Finished 35
Finished 36
Finished 37
Finished 38
Fini